In [1]:
import torch
import json
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer
)
from peft import PeftModel

/common/home/za224/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# The Base Model name
base_model_name = "meta-llama/Llama-2-7b-chat-hf"

# Fine-tuned model adapter path
ft_model = "qlora-adapters/llama-2-7b-qlora/"

In [3]:
def get_auth_token():
  token_file = "auth_token.txt"
  with open(token_file) as f:
    auth_token = f.read()
    return auth_token

In [4]:
# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_name, use_auth_token=get_auth_token())
tokenizer.pad_token = "<PAD>"
tokenizer.padding_side = "right"

/common/home/za224/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1714: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [5]:
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map={"": 0},
    use_auth_token=get_auth_token()
)

Loading checkpoint shards:  50%|█████     | 1/2 [00:02<00:02,  2.49s/it]


OutOfMemoryError: CUDA out of memory. Tried to allocate 86.00 MiB. GPU 0 has a total capacty of 10.91 GiB of which 74.12 MiB is free. Including non-PyTorch memory, this process has 10.83 GiB memory in use. Of the allocated memory 10.37 GiB is allocated by PyTorch, and 1.57 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
model = PeftModel.from_pretrained(base_model, ft_model)
model = model.merge_and_unload()

In [ ]:
merged_model_path = "finetuned-models/lama-2-7b-chat-custom" 

# Save the merged model
model.save_pretrained(merged_model_path)
tokenizer.save_pretrained(merged_model_path)